In [64]:
import numpy as np
import math
from tabulate import tabulate
import matplotlib.pyplot as plt
import pandas as pd
import sympy as sp

In [65]:
j = 12
n = 10
a = 0.7
b = 1.7
alpha_j = 0.1 + 0.05*j
h = 1/n

def f(x):
    return alpha_j * math.exp(x) + (1 - alpha_j) * math.sin(x)

In [66]:
x_vals = np.array(sorted([(a+b) / 2 + (b-a) / 2 * math.cos(math.pi * (2*i+1) / (2*n+2)) for i in range(n+1)]))
f_vals = np.array([f(x_) for x_ in x_vals])
x_star = np.array([x_vals[0] + 2*h/3, x_vals[n // 2] + h / 2, x_vals[-1] - h / 3])
f_star = np.array([f(x_) for x_ in x_star])
delta_x_star = [min([abs(x - x_s) for x in x_vals]) for x_s in x_star]

print(tabulate(zip(x_vals, f_vals), headers=['x', 'f(x)']))
print('\nСпециальные точки')
print(tabulate(zip(x_star, f_star), headers=['x*', 'f(x*)']))
print('\nМинимальное расстояние от контрольных точек до узлов интерполяции:\n')
print(tabulate(zip(x_star, delta_x_star), headers=['x*_i', '\delta_i']))

       x     f(x)
--------  -------
0.705089  1.61125
0.745184  1.67821
0.822125  1.81251
0.92968   2.01402
1.05913   2.28029
1.2       2.60369
1.34087   2.96775
1.47032   3.34393
1.57787   3.69125
1.65482   3.96142
1.69491   4.11

Специальные точки
      x*    f(x*)
--------  -------
0.771756  1.72371
1.25      2.72794
1.66158   3.98609

Минимальное расстояние от контрольных точек до узлов интерполяции:

    x*_i    \delta_i
--------  ----------
0.771756  0.0265719
1.25      0.05
1.66158   0.00676139


In [67]:
# Таблица значений функции
table = pd.DataFrame({"x_i": x_vals, "f(x_i)": f_vals})
table_transposed = table.T

# Точки для проверки интерполяции
x_star1 = x_star[0]
x_star2 = x_star[1]
x_star3 = x_star[2]

f_x_star = f_star[0]
f_x_star2 = f_star[1]
f_x_star3 = f_star[2]

def compute_newton_coefficients(x_vals, y_vals):
    """
    Возвращает список коэффициентов интерполяционного многочлена Ньютона
    с использованием рекурсивного определения разделённых разностей.
    """
    n = len(x_vals)
    # Создаём таблицу размером n x n
    dd_table = [y_vals.copy()]  # f[x_i]

    for level in range(1, n):
        prev_column = dd_table[-1]
        curr_column = []
        for i in range(n - level):
            numerator = prev_column[i + 1] - prev_column[i]
            denominator = x_vals[i + level] - x_vals[i]
            curr_column.append(numerator / denominator)
        dd_table.append(curr_column)

    # Коэффициенты Ньютона — это верхние элементы каждого столбца
    return dd_table, [dd_table[i][0] for i in range(n)]
    
dd_table, newton_coeffs = compute_newton_coefficients(x_vals, f_vals)


def newton_interpolation(x_vals, y_vals, x, coef):
    """
    Вычисляет значение интерполяционного многочлена Ньютона в точке x
    с использованием рекурсивной формулы:
    P_{n+1}(x) = P_n(x) + alpha_{n+1} * omega_{n+1}(x)
    """
    result = coef[0]
    omega = 1.0
    for i in range(1, len(coef)):
        omega *= (x - x_vals[i - 1])
        result += coef[i] * omega
    return result, omega

omegas = np.array([1.0,1.0,1.0])
P_x_star, omegas[0] = newton_interpolation(x_vals, f_vals, x_star1, newton_coeffs)
P_x_star2, omegas[1] = newton_interpolation(x_vals, f_vals, x_star2, newton_coeffs)
P_x_star3, omegas[2] = newton_interpolation(x_vals, f_vals, x_star3, newton_coeffs)

omega_frame = pd.DataFrame(omegas, index=[f'omega_{i}' for i in range(len(omegas))])
# Результаты интерполяции
data = {
    "Точка": ["x*", "x**", "x***"],
    "Значение x": [x_star1, x_star2, x_star3],
    "f(x)": [f_x_star, f_x_star2, f_x_star3],
    "P(x) (полином)": [P_x_star, P_x_star2, P_x_star3]
}

#n = len(newton_coeffs)
dd_frame = pd.DataFrame(dd_table, columns=[f"f[x0..x{i}]" for i in range(len(newton_coeffs))])
dd_frame.insert(0, "x_i", x_vals)
coeff_frame = pd.DataFrame(newton_coeffs, index=[f"a_{i}" for i in range(len(newton_coeffs))]).T
df = pd.DataFrame(data)

# Производная (n+1)-го порядка
x = sp.Symbol('x')
f_sym = alpha_j * sp.exp(x) + (1 - alpha_j) * sp.sin(x)
f_derivative = sp.diff(f_sym, x, n + 1)

# Максимум абсолютного значения производной на отрезке [0.7, 1.7]
f_derivative_abs = sp.lambdify(x, sp.Abs(f_derivative), 'numpy')
x_test = np.linspace(a, b, 1000)
M_max = np.max(f_derivative_abs(x_test))

# Истинная погрешность
r_x_star = f_x_star - P_x_star
r_x_star2 = f_x_star2 - P_x_star2
r_x_star3 = f_x_star3 - P_x_star3

# Оценка погрешности по неравенству
factorial = math.factorial(n + 1)
x_stars = [x_star1, x_star2, x_star3]
r_x_stars = [r_x_star, r_x_star2, r_x_star3]
error_bound_stars_v1 = [M_max / factorial * 2 * ((b-a)/4)**(n+1) for i in range(3)]
error_bound_stars_v2 = [dd_table[0][n-1] * 2 * ((b-a)/4)**(n+1) for i in range(3)]

#ab_pow = ((b-a)/4)**(n+1)
# for i in range(len(omegas)):
# #    prod_term = np.prod([abs(x_val - xi) for xi in x_vals])
#     error_bound = M_max / factorial * 2 * ab_pow
#     error_bound_stars.append(error_bound)

# Проверка выполнения неравенства
is_error_bound_stars_valid_v1 = [
    abs(r_x_stars[i]) <= error_bound_stars_v1[i] for i in range(3)
]
is_error_bound_stars_valid_v2 = [
    abs(r_x_stars[i]) <= error_bound_stars_v2[i] for i in range(3)
]

# Таблица ошибок
error_table = pd.DataFrame({
    "Точка": ["x*", "x**", "x***"],
    "Значение x": [x_star1, x_star2, x_star3],
    "r истинная": [abs(r) for r in r_x_stars],
    "оценка погрешности v1": error_bound_stars_v1,
    "оценка погрешности v2": error_bound_stars_v2,
    "M = max|f^(n+1)(x)|": [M_max] * 3,
    "Неравенство оценки v1 выполняется?": is_error_bound_stars_valid_v1,
    "Неравенство оценки v2 выполняется?": is_error_bound_stars_valid_v2

})

# Вывод таблиц
display(table_transposed)
display(df)
display(error_table)
display(dd_frame)
display(coeff_frame)
display(omega_frame)


,0,1,2,3,4,5,6,7,8,9,10
x_i,0.705089,0.745184,0.822125,0.929680,1.059134,1.200000,1.340866,1.470320,1.577875,1.654816,1.694911
f(x_i),1.611250,1.678212,1.812509,2.014017,2.280290,2.603694,2.967752,3.343927,3.691247,3.961424,4.110004


,Точка,Значение x,f(x),P(x) (полином)
0,x*,0.771756,1.723712,1.723712
1,x**,1.250000,2.727935,2.727935
2,x***,1.661577,3.986094,3.986094


,Точка,Значение x,r истинная,оценка погрешности v1,оценка погрешности v2,M = max|f^(n+1)(x)|,Неравенство оценки v1 выполняется?,Неравенство оценки v2 выполняется?
0,x*,0.771756,2.442491e-14,4.623513e-14,0.000002,3.870417,True,True
1,x**,1.250000,2.442491e-14,4.623513e-14,0.000002,3.870417,True,True
2,x***,1.661577,1.065814e-14,4.623513e-14,0.000002,3.870417,True,True


,x_i,f[x0..x0],f[x0..x1],f[x0..x2],f[x0..x3],f[x0..x4],f[x0..x5],f[x0..x6],f[x0..x7],f[x0..x8],f[x0..x9],f[x0..x10]
0,0.705089,1.611250e+00,1.678212,1.812509,2.014017,2.280290,2.603694,2.967752,3.343927,3.691247,3.961424,4.110004
1,0.745184,1.670114e+00,1.745449,1.873538,2.056892,2.295822,2.584426,2.905857,3.229246,3.511473,3.705735,NaN
2,0.822125,6.436914e-01,0.694265,0.773618,0.883879,1.024389,1.189075,1.364462,1.529719,1.659849,NaN,NaN
3,0.929680,2.251814e-01,0.252756,0.291793,0.341718,0.400514,0.464142,0.526380,0.579408,NaN,NaN,NaN
4,1.059134,7.788432e-02,0.085831,0.096242,0.108752,0.122659,0.136843,0.149778,NaN,NaN,NaN,NaN
5,1.200000,1.605737e-02,0.017478,0.019298,0.021456,0.023811,0.026136,NaN,NaN,NaN,NaN,NaN
6,1.340866,2.233963e-03,0.002511,0.002855,0.003248,0.003657,NaN,NaN,NaN,NaN,NaN,NaN
7,1.470320,3.618261e-04,0.000413,0.000472,0.000534,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.577875,5.868894e-05,0.000065,0.000071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1.654816,6.461712e-06,0.000007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,a_10
0,1.61125,1.670114,0.643691,0.225181,0.077884,0.016057,0.002234,0.000362,0.000059,0.000006,5.671354e-07


,0
omega_0,4.907741e-07
omega_1,9.560534e-07
omega_2,5.198465e-06
